In [54]:
import pandas as pd
import numpy as np
import joblib
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

In [ ]:
csv_path = "Dataset/hybrid.csv"   
target_col = "Carbon_Emissions"
timestamp_col = "Timestamp"
use_time_split = True                   
create_lag_features = False             
lag_hours = 1                           
test_fraction = 0.2
model_path = "co2_lgbm_model.pkl"

In [ ]:

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = np.abs(y_true) > 1e-9
    if not mask.any():
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / np.abs(y_true[mask]))) * 100

def print_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape_v = mape(y_true, y_pred)
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²:   {r2:.4f}")
    print(f"MAPE: {mape_v:.2f}%")
    return {"mae": mae, "rmse": rmse, "r2": r2, "mape": mape_v}

In [ ]:

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV not found at {csv_path}. Update csv_path variable.")

df = pd.read_csv(csv_path)

if timestamp_col in df.columns:
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors="coerce")
else:
    print("Warning: Timestamp column not found. Proceeding without time-based split.")
    use_time_split = False


print("Columns found:", df.columns.tolist())

if target_col not in df.columns:
    raise ValueError(f"Target column '{target_col}' not found in CSV.")

df = df.replace({"Yes": 1, "No": 0})

df = df.dropna(subset=[target_col])

if create_lag_features and timestamp_col in df.columns:
    df = df.sort_values(timestamp_col).reset_index(drop=True)
    for lag in range(1, lag_hours + 1):
        df[f"co2_lag_{lag}"] = df[target_col].shift(lag)
    df = df.dropna().reset_index(drop=True)


feature_cols = [c for c in df.columns if c not in [target_col, timestamp_col]]
print("Features used:", feature_cols)


if len(df) < 30:
    print("Warning: dataset has fewer than 30 rows; model performance may be poor.")

Columns found: ['Timestamp', 'Energy_Consumption', 'Material_Usage', 'Waste_Generated', 'Production_Capacity', 'Reconfiguration_Time', 'Downtime', 'Machine_Load', 'Temperature', 'Renewable_Usage', 'AI_Optimization_Applied', 'IoT_Enabled', 'Predictive_Maintenance', 'Digital_Twin_Used', 'Optimum_Value_for_Tool_Change', 'Optimum_Value_for_Setup_Change', 'Carbon_Emissions']
Features used: ['Energy_Consumption', 'Material_Usage', 'Waste_Generated', 'Production_Capacity', 'Reconfiguration_Time', 'Downtime', 'Machine_Load', 'Temperature', 'Renewable_Usage', 'AI_Optimization_Applied', 'IoT_Enabled', 'Predictive_Maintenance', 'Digital_Twin_Used', 'Optimum_Value_for_Tool_Change', 'Optimum_Value_for_Setup_Change']


In [ ]:

if use_time_split and timestamp_col in df.columns:
    df = df.sort_values(timestamp_col).reset_index(drop=True)
    split_idx = int(len(df) * (1 - test_fraction))
    train_df = df.iloc[:split_idx]
    test_df = df.iloc[split_idx:]

    X_train = train_df[feature_cols]
    y_train = train_df[target_col]
    X_test = test_df[feature_cols]
    y_test = test_df[target_col]

    print(f"Time-based split: train={len(X_train)} rows, test={len(X_test)} rows")

else:
    X = df[feature_cols]
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_fraction, random_state=42
    )
    print(f"Random split: train={len(X_train)} rows, test={len(X_test)} rows")

model = LGBMRegressor(
    objective="regression",
    n_estimators=3000,
    learning_rate=0.01,
    num_leaves=256,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)


try:
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        callbacks=[early_stopping(stopping_rounds=50), log_evaluation(period=0)]
    )
except TypeError:
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=50,
        verbose=False
    )

print("Model trained.")

Time-based split: train=168422 rows, test=42106 rows
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1253
[LightGBM] [Info] Number of data points in the train set: 168422, number of used features: 15
[LightGBM] [Info] Start training from score 205.397590
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1658]	valid_0's l2: 9.61293
✅ Model trained.


In [ ]:

joblib.dump(model, model_path)
print(f"Model saved to {model_path}")

Model saved to co2_lgbm_model.pkl


In [ ]:

y_pred_test = model.predict(X_test)
print("\nTest set performance:")
metrics = print_metrics(y_test, y_pred_test)

fi = pd.DataFrame({
    "feature": feature_cols,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)
print("\nFeature importances (top 10):")
print(fi.head(10).to_string(index=False))


Test set performance:
MAE:  2.4748
RMSE: 3.1005
R²:   0.9863
MAPE: 1.23%

Feature importances (top 10):
                       feature  importance
               Renewable_Usage       52073
                Material_Usage       47204
                  Machine_Load       44352
                   Temperature       42781
            Energy_Consumption       41619
                      Downtime       31693
           Production_Capacity       29413
          Reconfiguration_Time       22676
Optimum_Value_for_Setup_Change       19855
 Optimum_Value_for_Tool_Change       17680


In [ ]:

def apply_percent_changes(base_df, percent_changes):
    """
    percent_changes: dict like {"Energy_Consumption": -0.2, "Waste_Generated": -0.1}
    negative means reduce by 20%, positive means increase by e.g. 0.1 -> +10%
    base_df: pd.DataFrame with a single row (sample)
    returns modified_df
    """
    mod = base_df.copy().astype(float)
    for col, pct in percent_changes.items():
        if col not in mod.columns:
            raise KeyError(f"{col} not in input features")
        mod[col] = mod[col] * (1.0 + float(pct))
    return mod

def apply_absolute_changes(base_df, absolute_changes):
    """
    absolute_changes: dict like {"Energy_Consumption": -20} to subtract 20 units
    """
    mod = base_df.copy().astype(float)
    for col, delta in absolute_changes.items():
        if col not in mod.columns:
            raise KeyError(f"{col} not in input features")
        mod[col] = mod[col] + float(delta)
    return mod

def run_what_if(model, base_sample, percent_changes=None, absolute_changes=None):
    """
    base_sample: DataFrame with single row containing feature_cols
    returns dict with base_pred, new_pred, reduction, pct_reduction
    """
    if isinstance(base_sample, pd.Series):
        base_sample = base_sample.to_frame().T
    base_sample = base_sample[feature_cols].reset_index(drop=True)
    base_pred = float(model.predict(base_sample)[0])

    modified = base_sample.copy()
    if percent_changes:
        modified = apply_percent_changes(modified, percent_changes)
    if absolute_changes:
        modified = apply_absolute_changes(modified, absolute_changes)

    new_pred = float(model.predict(modified)[0])
    reduction = base_pred - new_pred
    pct_reduction = (reduction / base_pred * 100) if base_pred != 0 else np.nan

    return {
        "base_pred": base_pred,
        "new_pred": new_pred,
        "reduction_abs": reduction,
        "reduction_pct": pct_reduction,
        "modified_input": modified.iloc[0].to_dict()
    }

def run_batch_scenarios(model, base_sample, scenarios):
    """
    scenarios: dict name->percent_changes dict
    returns DataFrame summarizing scenarios
    """
    rows = []
    for name, pct_changes in scenarios.items():
        res = run_what_if(model, base_sample, percent_changes=pct_changes)
        rows.append({
            "scenario": name,
            "base_pred": res["base_pred"],
            "new_pred": res["new_pred"],
            "reduction_abs": res["reduction_abs"],
            "reduction_pct": res["reduction_pct"]
        })
    return pd.DataFrame(rows)

In [ ]:

base_sample = X_test.iloc[[0]] 
print("\nBase sample (features):")
print(base_sample.to_string(index=False))

base_pred = float(model.predict(base_sample)[0])
print(f"\nBase predicted CO₂: {base_pred:.3f}")

scenario = {"Energy_Consumption": -0.20}
res = run_what_if(model, base_sample, percent_changes=scenario)
print("\nWhat-if scenario (Energy -20%):")
print(f"Predicted CO₂ (new): {res['new_pred']:.3f}")
print(f"Absolute reduction: {res['reduction_abs']:.3f}")
print(f"Percent reduction: {res['reduction_pct']:.2f}%")

scenarios = {
    "Energy -20%": {"Energy_Consumption": -0.20},
    "Waste -15%": {"Waste_Generated": -0.15},
    "Prod -10%": {"Production_Capacity": -0.10},
    "Energy -20% + Waste -10%": {"Energy_Consumption": -0.20, "Waste_Generated": -0.10}
}
batch_df = run_batch_scenarios(model, base_sample, scenarios)
print("\nBatch scenario results:")
print(batch_df.to_string(index=False))



Base sample (features):
 Energy_Consumption  Material_Usage  Waste_Generated  Production_Capacity  Reconfiguration_Time  Downtime  Machine_Load  Temperature  Renewable_Usage  AI_Optimization_Applied  IoT_Enabled  Predictive_Maintenance  Digital_Twin_Used  Optimum_Value_for_Tool_Change  Optimum_Value_for_Setup_Change
                164             271               11                  153                    32        10            73        15.86            28.45                        0            1                       0                  1                             19                              43

Base predicted CO₂: 174.368

What-if scenario (Energy -20%):
Predicted CO₂ (new): 158.023
Absolute reduction: 16.345
Percent reduction: 9.37%

Batch scenario results:
                scenario  base_pred   new_pred  reduction_abs  reduction_pct
             Energy -20% 174.367991 158.022683      16.345308       9.374030
              Waste -15% 174.367991 174.409584      -0.041593    

In [ ]:
def simulate_real_time_predictions(model, cycles=10, sleep_seconds=2):
    print("\nStarting simulated real-time predictions...")
    for i in range(cycles):
        rand_row = X_train.sample(n=1, replace=True).reset_index(drop=True)
        pred = float(model.predict(rand_row)[0])
        ts = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{ts}] Predicted CO2: {pred:.2f} | sample: {rand_row.to_dict(orient='records')[0]}")
       
        import time
        time.sleep(sleep_seconds)


simulate_real_time_predictions(model, cycles=5, sleep_seconds=1)


print("\nDone. You can reuse model at:", model_path)


Starting simulated real-time predictions...
[2025-09-05 15:52:27] Predicted CO2: 172.42 | sample: {'Energy_Consumption': 171, 'Material_Usage': 283, 'Waste_Generated': 19, 'Production_Capacity': 159, 'Reconfiguration_Time': 35, 'Downtime': 5, 'Machine_Load': 50, 'Temperature': 28.86, 'Renewable_Usage': 29.7, 'AI_Optimization_Applied': 1, 'IoT_Enabled': 1, 'Predictive_Maintenance': 1, 'Digital_Twin_Used': 0, 'Optimum_Value_for_Tool_Change': 27, 'Optimum_Value_for_Setup_Change': 49}
[2025-09-05 15:52:28] Predicted CO2: 245.70 | sample: {'Energy_Consumption': 230, 'Material_Usage': 323, 'Waste_Generated': 23, 'Production_Capacity': 191, 'Reconfiguration_Time': 30, 'Downtime': 25, 'Machine_Load': 88, 'Temperature': 31.79, 'Renewable_Usage': 28.09, 'AI_Optimization_Applied': 0, 'IoT_Enabled': 0, 'Predictive_Maintenance': 0, 'Digital_Twin_Used': 0, 'Optimum_Value_for_Tool_Change': 23, 'Optimum_Value_for_Setup_Change': 27}
[2025-09-05 15:52:29] Predicted CO2: 245.80 | sample: {'Energy_Consum